In [3]:
import sys#THIS IS FOR GOOGLE COLAB USERS ONLY
from google.colab import files
sys.path.insert(0,'/content/drive/MyDrive/Product-Title-Classification')

In [5]:
import sys                                                                  #THIS IS FOR GOOGLE COLAB USERS ONLY
from google.colab import files
sys.path.insert(0,'/content/drive/MyDrive/Product-Title-Classification')
!cp /content/drive/MyDrive/Product-Title-Classification/Utilities.py /content           #THIS IS FOR GOOGLE COLAB USERS ONLY
!cp /content/drive/MyDrive/Product-Title-Classification/KNNImpute.py /content

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importing Local Files

In [7]:
import KNNImpute
import Utilities

Importing Directories

In [45]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Splitting Data Into Training Data and Testing Data

In [ ]:
labels=["country", "sku_id", "title", "category_lvl1","category_lvl2","category_lvl3", "description", "price", "type"]

def getTrainingDataset():
    return pd.read_csv('data_train.csv', header=None, names=labels)  


raw_data = getTrainingDataset()
train_df,Y1,Y2,Y3=Utilities.Cleaning_Data_Utility(raw_data) #Utilities. hatadena yahan se 
print(train_df.head())

Imputing Null Values

In [ ]:
def NullStatistics(df):
    missing_val = df.isnull().sum()
    print(missing_val)
    total_cells = np.product(df.shape)
    missing_percent = (missing_val.sum()/total_cells) *100
    print(f'\nThe missing data percent is: {missing_percent}')

NullStatistics(train_df)

unique_label_c1, unique_label_c2, unique_label_c3 = KNNImpute.preserve_label(train_df) #It has all unique values lying in ctg1 , 2 , 3 column
KNNImpute.encode(['category_lvl1', 'category_lvl2', 'category_lvl3'],train_df)        #Performed encoding for CTGLVL3 KNN
train_df_imputed = KNNImpute.impute(train_df)
train_df_imputed =  KNNImpute.clean_csv(train_df_imputed,train_df)
NullStatistics(train_df_imputed)

Splitting Data Into Training And Testing

In [93]:
X = train_df_imputed['Title_desc']
Y1 = train_df_imputed['category_lvl1']
Y2 = train_df_imputed['category_lvl2']
Y3 = train_df_imputed['category_lvl3']
X_train, X_test , Y_train , Y_test = train_test_split(X, Y1, test_size = 0.2, random_state=3 , stratify = Y1)

In [94]:
X_train

4353     exceed multi chain bead tassel pendant chain n...
11016    wireless bluetooth headset stereo earphon spor...
29446    eyebrow style ul li great beauti tool make eye...
28291    okdeal nail art rhineston glitter tip manicur ...
3603     joyroom fast charg metal data snyc pin usb cab...
                               ...                        
11210    new women dress print pattern floral parti dre...
31349    assassin creed men shirt male assassin short s...
16163    zanzea women plu long sleev knit casual loos b...
30263    heavi duti rug hybrid dual layer kickstand sho...
36042    ufengk uf wlj height chart decal multicolor ul...
Name: Title_desc, Length: 29026, dtype: object

Feature Extraction

In [95]:
from sklearn.feature_extraction.text import TfidfVectorizer

feature_extraction = TfidfVectorizer(min_df = 1, stop_words = 'english' , lowercase = 'True')

X_train_features = feature_extraction.fit_transform(X_train)
X_test_features = feature_extraction.transform(X_test)

In [96]:
print(X_train_features)

  (0, 27412)	0.05147738236103871
  (0, 21725)	0.1072819594650056
  (0, 11535)	0.08003847281153648
  (0, 14080)	0.27190821365232254
  (0, 25732)	0.034016658777698355
  (0, 22260)	0.13272624252479762
  (0, 16715)	0.7529840422150432
  (0, 18344)	0.26006979133219504
  (0, 24384)	0.2261839511670145
  (0, 2234)	0.19645108647174098
  (0, 4188)	0.341255582202283
  (0, 16384)	0.15400414309073085
  (0, 8333)	0.14893258931139913
  (1, 17900)	0.1187796477332564
  (1, 27021)	0.09095945706100005
  (1, 4773)	0.07144155218168634
  (1, 3475)	0.09541786037198048
  (1, 13995)	0.08633086231216248
  (1, 24316)	0.14876099646713956
  (1, 11561)	0.21184545778201172
  (1, 24905)	0.18212663945171972
  (1, 16466)	0.11824598591148702
  (1, 6044)	0.12187233769088548
  (1, 17809)	0.10713866420304043
  (1, 76)	0.11349203855719031
  :	:
  (29025, 4290)	0.19907250932268616
  (29025, 120)	0.18773123660120625
  (29025, 20326)	0.18576261391464532
  (29025, 4995)	0.14265540393097534
  (29025, 21985)	0.32373904813504456
  

Applying Random Forest Classifier

In [97]:
import math
X_train.shape
number_of_trees = X_train.shape
estimated_trees = math.sqrt(number_of_trees[0])
estimated_trees = int(math.floor(estimated_trees))
print(estimated_trees)

170


In [98]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators = estimated_trees) 
model.fit(X_train_features, Y_train)

RandomForestClassifier(n_estimators=170)

In [99]:
#Predicting On Training Data

prediction_on_training_data = model.predict(X_train_features)
accuracy_on_training_data = accuracy_score(Y_train , prediction_on_training_data)
print(accuracy_on_training_data)

0.9991387032315855


In [100]:
#Predicting On Testing Data

prediction_on_testing_data = model.predict(X_test_features)
accuracy_on_testing_data = accuracy_score(Y_test , prediction_on_testing_data)
print(accuracy_on_testing_data)

0.905057186165082


In [101]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix

print('Accuracy score with Random Forest Classifier is : ')
print(metrics.accuracy_score(Y_test, prediction_on_testing_data)*100)
mat = confusion_matrix(Y_test, prediction_on_testing_data)
print(mat)

Accuracy score with Random Forest Classifier is : 
90.5057186165082
[[ 340    8    2    1   21    1   14    2    1]
 [   2  498    1    1   18    2   35   27    1]
 [   0    1 1094    2   39    0    5    1    4]
 [   0    0   24  699   70    5    3    2    5]
 [   4    5   36   28 1100    5   19    2   11]
 [   0    1    2    3   75  233    3    0    0]
 [   4    4    2    1   12    2 1413   17    1]
 [   1   16    1    3   43    1   39  386   12]
 [   2    0   14    1   15    0    3    3  805]]


In [102]:
# Printing Classification Report

from sklearn.metrics import classification_report
print(classification_report(Y_test, prediction_on_testing_data))

              precision    recall  f1-score   support

         0.0       0.96      0.87      0.92       390
         1.0       0.93      0.85      0.89       585
         2.0       0.93      0.95      0.94      1146
         3.0       0.95      0.87      0.90       808
         4.0       0.79      0.91      0.85      1210
         5.0       0.94      0.74      0.82       317
         6.0       0.92      0.97      0.95      1456
         7.0       0.88      0.77      0.82       502
         8.0       0.96      0.95      0.96       843

    accuracy                           0.91      7257
   macro avg       0.92      0.88      0.89      7257
weighted avg       0.91      0.91      0.90      7257

